The following code load optimized deep convoluational neural network model for angle 1 ('resnet18.pth') and predict classes for sample 3D-surface images in a directory for the samples ('./sample_data/'). 'prediction_results.txt' includes prediction results for each 3D-surface images. 

In [1]:
#import modules
import torch
from torch.utils.data import DataLoader
from torch import nn
from torch.nn import functional as F
import torchvision
from torchvision import datasets, transforms
from torch import optim

import numpy as np
import pandas as pd
from PIL import Image, ImageOps
import os
import matplotlib as plt
import glob
import re
import sys

In [2]:
#image size setting
image_size = 224

#load the trained model
model_from_all = torch.load('resnet18.pth', map_location="cpu")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#check if the model is in inference mode. False indicates that the model is in inference mode.
print(model_from_all.training)

False


C:\Users\thase\anaconda3_2023\Lib\site-packages\torch\serialization.py:995: UserWarning: 'torch.load' received a zip file that looks like a TorchScript archive dispatching to 'torch.jit.load' (call 'torch.jit.load' directly to silence this warning)
  warnings.warn("'torch.load' received a zip file that looks like a TorchScript archive"


In [3]:
#list for index and prediction
index_list = []
prediction_list = []

In [4]:
#list up input image files wthinin a directory (in this case, ./sample_data/). 
ddir = './sample_data/*.JPG'
l = glob.glob(ddir)
print(l)

['./sample_data\\1423060_at_ML0-TCV_TTG037-L.JPG', './sample_data\\1434367_s_at_ML0-TCV_TTG118-L.JPG', './sample_data\\1439675_at_ML0-TCV_TTG129-L.JPG', './sample_data\\1460196_at_ML0-TCV_TTG131B-L.JPG', './sample_data\\1460258_at_ML0-TCV_TTG131B-L.JPG', './sample_data\\1460425_at_ML0-TCV_TTG131B-L.JPG']


In [5]:
#prediction
for i in l:
   img = Image.open(i)
   trans = transforms.Compose([
           transforms.Resize(image_size),
           transforms.ToTensor(),
       ])
   img = trans(img).unsqueeze(0)
   output = model_from_all(img.to(device))
   _, prediction = torch.max(output, 1)
   index_list.append(i)
   if prediction.cpu().numpy()[0] == 0:
      prediction_list.append('DOWN')
   elif prediction.cpu().numpy()[0] == 1:
      prediction_list.append('NS')
   elif prediction.cpu().numpy()[0] == 2:
      prediction_list.append('UP')

In [6]:
#output prediction results
report = pd.DataFrame()
report['file_name'] = index_list
report['prediction'] = prediction_list
report.to_csv("prediction_results.txt",sep="\t")
print(report)

                                         file_name prediction
0    ./sample_data\1423060_at_ML0-TCV_TTG037-L.JPG         UP
1  ./sample_data\1434367_s_at_ML0-TCV_TTG118-L.JPG         UP
2    ./sample_data\1439675_at_ML0-TCV_TTG129-L.JPG         UP
3   ./sample_data\1460196_at_ML0-TCV_TTG131B-L.JPG         UP
4   ./sample_data\1460258_at_ML0-TCV_TTG131B-L.JPG         UP
5   ./sample_data\1460425_at_ML0-TCV_TTG131B-L.JPG         UP
